# 1. Install & Import 라이브러리

In [8]:
!pip3 install selenium webdriver_manager beautifulsoup4 pandas

from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
import time
import os
from bs4 import BeautifulSoup
from time import sleep
import pandas as pd
import json


Defaulting to user installation because normal site-packages is not writeable
  Using cached webdriver_manager-4.0.1-py2.py3-none-any.whl.metadata (12 kB)
  Using cached python_dotenv-1.0.1-py3-none-any.whl.metadata (23 kB)
Using cached webdriver_manager-4.0.1-py2.py3-none-any.whl (27 kB)
Using cached python_dotenv-1.0.1-py3-none-any.whl (19 kB)


Please see https://github.com/pypa/pip/issues/5599 for advice on fixing the underlying issue.
To avoid this problem you can invoke Python with '-m pip' instead of running pip directly.


# 2. 경로 및 URL설정

In [2]:
PATH_POST_DATA_CSV = 'data_csv/post.csv'
PATH_POST_DATA_JSON = 'data_json/post.json'

URL_BASE = 'https://gall.dcinside.com/board/lists/?id=football_new8&page='

# 3. 함수 설정

In [3]:
def reading_file(file_path):
    encodings = ['cp949', 'utf-8', 'ISO-8859-1', 'cp1252']  # List of encodings to try
    for encoding in encodings:
        try:
            df = pd.read_csv(file_path, encoding=encoding)
            return df  # or process your df
        except: pass
    return None


# 4. 크롤링 (해외축구 갤러리 게시글)

In [ ]:
# 게시글 데이터 파일 불러오기
with open(PATH_POST_DATA_JSON, 'r', encoding='utf-8') as f:
    data_post = json.load(f)

# 해외축구갤러리 크롤링 -> 게시글 타이틀
def crawling_data():
    # 웹 드라이버 초기화
    options = webdriver.ChromeOptions()
    service = Service(ChromeDriverManager().install())
    driver = webdriver.Chrome(service=service, options=options)

    # 설정한 페이지 크기만큼 반복
    max_page = 100001  
    for page in range(1, max_page)[::100]:
        # 진행률
        progress = f'{((page+1) / max_page) * 100:.3f}'
        print(f'\r{progress}%, {page}', end ='')
        
        # 페이지 접속
        try:
            driver.get(f'{URL_BASE}{page}')  
            sleep(3) # 로딩 대기
        except Exception as e: 
            print(e)
            continue

        # BeautifulSoup 객체 생성
        page_source = driver.page_source
        soup = BeautifulSoup(page_source, "html.parser")  

        # 페이지 내 게시물 정보 추출
        for tr in soup.find_all('tr', {'class':'ub-content us-post'}):
            try:
                # 게시물 내용 추출
                date = tr.find('td', {'class':'gall_date'})['title']
                content = tr.find('a').text.strip('\n')
                
                # 변경사항을 JSON 파일에 바로 저장
                if not date in data_post:
                    data_post[date] = [date, content]
                    with open(PATH_POST_DATA_JSON, 'w', encoding='utf-8') as f: 
                        json.dump(data_post, f)
            except Exception as e:
                print(e)
                pass

# CSV 파일이 없으면 크롤링 시작
if os.path.exists(PATH_POST_DATA_CSV) == False:
    # 데이터 크롤링
    crawling_data()
    
    # 데이터 포맷 조정
    data_box = []
    for key in data_post:
        date, post = data_post[key]
        data_box.append([date, post])

    # DataFrame 생성 후 CSV로 저장
    df_post = pd.DataFrame(list(data_post.values()), columns=['DATE', 'CONTENT'])
    df_post.to_csv(PATH_POST_DATA_CSV, index=False)
else:
    # CSV 파일이 이미 존재하면 파일에서 데이터 읽기
    df_post = reading_file(PATH_POST_DATA_CSV)

df_post  # 최종 데이터 프레임 출력
